https://future-chem.com/
こちらよりスクリプトを引用しました。

In [1]:
import psi4

psi4.set_memory('500 MB')

h2o = psi4.geometry("""
O
H 1 0.96
H 1 0.96 2 104.5
""")

psi4.energy('scf/cc-pvdz')

-76.02663273509032

In [2]:
import datetime
import time

t = datetime.datetime.fromtimestamp(time.time())

psi4.set_num_threads(nthread=12)
psi4.set_memory('16GB')
psi4.set_output_file('{}{}{}_{}{}.log'.format(t.year,
                                              t.month,
                                              t.day,
                                              t.hour,
                                              t.minute))

In [3]:
### Z-matrix
h2O_zmat = psi4.geometry('''
0 1
O
H 1 0.96
H 1 0.96 2 104.5''')

In [4]:
theory = ['hf', 'b3lyp']
basis_set = ['sto-3g', '3-21G']

import itertools

for th, ba in itertools.product(theory, basis_set):
    level = th + '/' + ba
    e = psi4.energy(level, molecule=h2O_zmat)
    print('energy at the {} level of theory:\t{:.4f}'.format(level, e))

energy at the hf/sto-3g level of theory:	-74.9634
energy at the hf/3-21G level of theory:	-75.5854
energy at the b3lyp/sto-3g level of theory:	-75.3135
energy at the b3lyp/3-21G level of theory:	-75.9718


In [5]:
geom = '''
0 1
O
H 1 {0}
H 1 {0} 2 104.5'''

for i in [0.85, 0.90, 0.95, 1.00, 1.05]:
    h2O = psi4.geometry(geom.format(i))
    e = psi4.energy('hf/sto-3g', molecule=h2O)
    print('energy for d = {}: {:.4f}'.format(i, e))

energy for d = 0.85: -74.9100
energy for d = 0.9: -74.9451
energy for d = 0.95: -74.9618
energy for d = 1.0: -74.9648
energy for d = 1.05: -74.9572


In [5]:
import sys
import os
import datetime

file_name = sys.argv[0]
file, _ = os.path.splitext(file_name)
now = datetime.datetime.now()

logfile = file + '_{}{}{}_{}{}.log'.format(now.year, now.month, now.day, now.hour, now.minute)

psi4.set_num_threads(nthread=12)
psi4.set_memory('16GB')
psi4.set_output_file(logfile)

m_xylene = psi4.geometry('''
0 1
H          1.28968       -0.58485        2.54537
C          0.72665       -0.53821        1.60812
C         -0.66059       -0.63788        1.62278
H         -1.18866       -0.76325        2.57379
C         -1.38281       -0.57923        0.43824
H         -2.47598       -0.65808        0.46597
C         -0.70870       -0.41532       -0.78014
C         -1.44994       -0.24691       -1.99137
C          0.68999       -0.31852       -0.79417
H          1.23196       -0.19170       -1.73873
C          1.39668       -0.37916        0.39958
C          2.48879       -0.30069        0.38763
H         -2.49493       -0.35404       -1.78784
H         -1.14694       -0.98824       -2.70096
H         -1.26259        0.72757       -2.39162
H          2.86211       -0.36704        1.38820
H          2.77426        0.63858       -0.03801
H          2.89720       -1.09694       -0.19896
''')

psi4.optimize('hf/sto-3g', molecule=m_xylene)

Optimizer: Optimization complete!


-305.0608618365619

In [ ]:
anti = psi4.geometry('''
0 1
C         -4.17385        0.37732        0.00000
C         -3.10385        0.37732        0.00000
H         -4.53051        1.24338       -0.51734
H         -4.53052        0.39232        1.00869
C         -4.53052       -0.50373       -0.49135
C         -2.74719        1.25838        0.49135
H         -2.74718        0.36232       -1.00869
H         -2.74718       -0.48873        0.51734
H         -3.10386        2.12443       -0.02599
H         -1.67719        1.25838        0.49135
H         -3.10385        1.27338        1.50005
H         -4.17385       -0.51873       -1.50005
H         -5.60052       -0.50373       -0.49135
H         -4.17386       -1.36979        0.02599
''')

gauche = psi4.geometry('''
0 1
C         -2.25507        0.85394        0.00000
C         -1.18507        0.85394        0.00000
C         -2.61173        1.26615       -0.92075
H         -2.61173        1.44522        0.81736
H         -2.61174       -0.14955        0.10339
C         -0.82840        1.85743       -0.10339
H         -0.82840        0.26265       -0.81736
H         -0.82840        0.44173        0.92074
H         -1.18508        2.26964       -1.02414
H          0.24160        1.85744       -0.10339
H         -1.18507        2.44872        0.71397
H         -2.25506        2.26964       -1.02413
H         -3.68173        1.26615       -0.92075
H         -2.25506        0.67486       -1.73810
''')

e_anti = psi4.optimize('hf/sto-3g', molecule=anti)
e_gauche = psi4.optimize('hf/sto-3g', molecule=gauche)

print('Anti conformer: {:.5f}\tGauche conformer: {:.5f}'.format(e_anti, e_gauche))

xyzファイルを書き出す際に必要なスピン、電荷設定の一部自動化

In [6]:
## ライブラリのインポート
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import time

from rdkit import rdBase, Chem
from rdkit.Chem import AllChem, Draw, PandasTools
from rdkit.Chem.Draw import IPythonConsole

def spincount(smiles):
    txt = smiles
    spin = 1
    cat = txt.count("+")
    ani = txt.count("-")
    elec = cat - ani
    return(str(elec) + " " + str(spin))

spincount("C[NH+](C)C1CSSC1.C(=O)(C(=O)[O-])O")
    

'0 1'

smilesをxyzに書き出す

In [7]:
## ライブラリのインポート
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import time

from rdkit import rdBase, Chem
from rdkit.Chem import AllChem, Draw, PandasTools
from rdkit.Chem.Draw import IPythonConsole

print('rdkit version: ', rdBase.rdkitVersion) # rdkit version:  2019.09.3
print('psi4 version: ', psi4.__version__) # psi4 version:  1.3.2
print('numpy version: ', np.__version__) # numpy version:  1.17.4

## SMILESをxyz形式に変換
def smi2xyz(smiles):
    mol = Chem.AddHs(Chem.MolFromSmiles(smiles))
    AllChem.EmbedMolecule(mol, AllChem.ETKDGv2())
    AllChem.MMFFOptimizeMolecule(mol)
    conf = mol.GetConformer(-1)
    
    xyz = spincount(smiles)
    for atom, (x,y,z) in zip(mol.GetAtoms(), conf.GetPositions()):
        xyz += '\n'
        xyz += '{}\t{}\t{}\t{}'.format(atom.GetSymbol(), x, y, z)
        
    return xyz

rdkit version:  2020.03.1
psi4 version:  1.3.2
numpy version:  1.18.1


In [8]:
import pandas as pd
dfm = pd.DataFrame(pd.read_csv('対象ファイルは自分で指定してください.csv'))
#SMILESキーの入った列を指定
sf = dfm["Canonical SMILES"]

4                     CCOP(=S)(OCC)SCCl
5               CCSC(=O)N(CC(C)C)CC(C)C
6    C[NH+](C)C1CSSC1.C(=O)(C(=O)[O-])O
Name: Canonical SMILES, dtype: object

基底関数と汎関数を色々変えて各種計算化学的記述子を算出する関数

In [9]:
def calq(sm):

    MeCl = psi4.geometry(smi2xyz(sm))
    basis_set = ['sto-3g', '3-21G', '6-31G(d)']
    import pandas as pd
    df = pd.DataFrame()

    psi4.set_output_file('MeCl.txt')
    psi4.energy('hf/sto-3g', molecule=MeCl)
    for basis in basis_set:
        _, wfn_cl = psi4.energy('hf/' + basis, molecule=MeCl, return_wfn=True)
        psi4.oeprop(wfn_cl, 'MULLIKEN_CHARGES')
        df['hf/'+basis+"dipole_x"] = psi4.variable('SCF DIPOLE X')
        df['hf/'+basis+"dipole_y"] = psi4.variable('SCF DIPOLE Y')
        df['hf/'+basis+"dipole_z"] = psi4.variable('SCF DIPOLE Z')
        df['hf/'+basis] = np.array(wfn_cl.atomic_point_charges())
        
    return(df)

原子毎に出す計算科学的記述子については統計値をとってまとめる

In [10]:
dft = pd.DataFrame()
for tet in sf:
    dd = pd.DataFrame()
    dd = calq(tet)
    dd = dd.describe()[2:]
#ddの次元は基底関数と汎関数を何種類試すかで変わるので、手動で設定が必要（いや丁寧に書けば自動化できるんですが）
    dd = pd.DataFrame(dd.values.reshape((1,72)))
    dft = dft.append(dd)

In [11]:
dft

,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
0,NaN,NaN,NaN,0.251207,4.540700e-16,2.270350e-16,0.000000e+00,0.495908,9.081400e-16,2.270350e-16,...,NaN,0.859876,2.475712,1.630059,3.265817,1.261900,2.206544,1.522225,3.337275,1.090287
0,NaN,NaN,NaN,0.118708,1.125537e-16,2.251074e-16,5.627686e-17,0.371784,1.125537e-16,4.502149e-16,...,NaN,0.190413,-0.326526,1.798885,0.196140,0.504151,-0.258988,1.969681,0.213075,0.596390
0,NaN,NaN,NaN,0.429415,0.000000e+00,0.000000e+00,2.262744e-16,0.874066,1.131372e-16,0.000000e+00,...,NaN,1.583269,0.519458,4.168622,1.903422,2.560699,0.873783,3.587551,1.725704,3.267541


In [11]:
#csvファイルに書き出し
dft.to_csv('名前は自由.csv', index = False)